In [1]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, StackingRegressor
from sklearn.multioutput import MultiOutputRegressor
from random import random
from sklearn.model_selection import cross_validate
import matplotlib
from waveform_methods import binning
import pandas as pd

In [2]:
file = uproot.open('12360.root')
file2 = uproot.open('12362.root')

In [3]:
Xmax1 = file['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file['MCPrimaryInfo']['longNumCharged'].array()
Depth1 = file['MCPrimaryInfo']['longDepth'].array()
chi2_1 = file['CurvatureOnlyParams']['chi2_time'].array()
red1 = file['MCPrimaryInfo']['ghRedChiSqr'].array()
mass1 = [1 for i in range(len(red1))]

good = []
count = 0
for i in range(len(Xmax1)):
    if (red1[i]<300 and chi2_1[i]<5) and Xmax1[i]<900:
        good.append(1)
        count+=1
    else:
        good.append(0)
        
Xmax2 = file2['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file2['MCPrimaryInfo']['longNumCharged'].array()
Depth2 = file2['MCPrimaryInfo']['longDepth'].array()
chi2_2 = file2['CurvatureOnlyParams']['chi2_time'].array()
red2 = file2['MCPrimaryInfo']['ghRedChiSqr'].array()
mass2 = [4 for i in range(len(red2))]

good2 = []
for i in range(len(Xmax2)):
    if (red2[i]<300 and chi2_2[i]<5) and Xmax2[i]<650:
        good2.append(1)
        count+=1
    else:
        good2.append(0)
print(count)

172382


In [4]:
S125_1 = np.log10(file['LaputopParams']['s125'].array())
S125_2 = np.log10(file2['LaputopParams']['s125'].array())

In [5]:
A1 = file['CurvatureOnlyParams']['A'].array()
A2 = file2['CurvatureOnlyParams']['A'].array()
D1 = file['CurvatureOnlyParams']['D'].array()
D2 = file2['CurvatureOnlyParams']['D'].array()
N1 = file['CurvatureOnlyParams']['N'].array()
N2 = file2['CurvatureOnlyParams']['N'].array()
beta1 = file['LaputopParams']['beta'].array()
beta2 = file2['LaputopParams']['beta'].array()
zenith1 = file['Laputop']['zenith'].array()
zenith2 = file2['Laputop']['zenith'].array()
rlogl1 = file['CurvatureOnlyParams']['rlogl'].array()
nmini1 = file['CurvatureOnlyParams']['nmini'].array()
ndf1 = file['CurvatureOnlyParams']['ndf'].array()
llh1 = file['CurvatureOnlyParams']['llh'].array()
rlogl2 = file2['CurvatureOnlyParams']['rlogl'].array()
nmini2 = file2['CurvatureOnlyParams']['nmini'].array()
ndf2 = file2['CurvatureOnlyParams']['ndf'].array()
llh2 = file2['CurvatureOnlyParams']['llh'].array()

Xmax1_new = Xmax1
Xmax2_new = Xmax2

In [6]:
A = np.log10(np.append(A1,A2))
D = np.append(D1,D2)
N = np.append(N1,N2)
S125 = np.append(S125_1,S125_2)
beta = np.log10(np.append(beta1,beta2))
chi2 = np.append(chi2_1,chi2_2)
zenith = np.cos(np.append(zenith1,zenith2))
Xmax = np.append(Xmax1_new,Xmax2_new)
goodness = np.append(good,good2)
red = np.append(red1,red2)
mass = np.append(mass1,mass2)

In [7]:
print(np.corrcoef(np.array([A,D,N,S125,beta,zenith,Xmax,chi2,red,mass,goodness])))

[[ 1.          0.11042152 -0.06389393  0.07506336  0.10801262  0.20482409
   0.02356671  0.08135272 -0.01841696 -0.07125706 -0.05894829]
 [ 0.11042152  1.         -0.0435993  -0.52563202  0.06855834  0.02395401
   0.00522173  0.06673223 -0.09536741 -0.14998897  0.0453969 ]
 [-0.06389393 -0.0435993   1.          0.08137576 -0.0860212  -0.10458563
  -0.01237658  0.42102965  0.04476834  0.0769891  -0.03354958]
 [ 0.07506336 -0.52563202  0.08137576  1.          0.07575187  0.19535921
   0.07220079 -0.05334439  0.17610724  0.04298084 -0.14193754]
 [ 0.10801262  0.06855834 -0.0860212   0.07575187  1.          0.41891977
   0.04970585 -0.02705426 -0.04467872 -0.20083333  0.15994521]
 [ 0.20482409  0.02395401 -0.10458563  0.19535921  0.41891977  1.
  -0.04087814 -0.0246141  -0.09956015  0.01339489  0.30870411]
 [ 0.02356671  0.00522173 -0.01237658  0.07220079  0.04970585 -0.04087814
   1.         -0.00294529  0.04444636 -0.19341011 -0.08236207]
 [ 0.08135272  0.06673223  0.42102965 -0.05334439

In [8]:
input_variable = np.array([np.append(np.append(i,j),k) for i,j,k in zip(S125,beta,zenith)])
output = list(zip(Xmax,goodness))

In [9]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.2
X_train, X_test, y_train_1, y_test_1 = train_test_split(input_variable,output , test_size=test_size, random_state=seed)

In [10]:
from scipy.stats import norm
y_train = list(zip(*y_train_1))[1]
y_test = list(zip(*y_test_1))[1]
for i in [5]:
    for j in [7]:
        rng = np.random.RandomState(1)
        regr_1 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=j,criterion='friedman_mse'),
                                n_estimators=i, random_state=rng,loss='square')
        regr_1.fit(X_train,y_train)
    
        y_1= regr_1.predict(X_test)
        y_2 = regr_1.predict(X_train)

        #mse = [(i-j)**2.0 for i,j in zip(y_1,y_test)]
        #mse2 = [(i-j)**2.0 for i,j in zip(y_2,y_train)]
        count_0 = 0
        count_02 = 0
        for value in y_train:
            if value==0:
                count_0+=1
        for value in y_test:
            if value==0:
                count_02+=1
        
        count = 0
        count2 = 0
        for value1,value2 in zip(y_1,y_test):
            if value1<0.5 and value2==0:
                count+=1

        for value1,value2 in zip(y_2,y_train):
            if value1<0.5 and value2==0:
                count2+=1

        print(count/count_02,count2/count_0,i,j)
        
        #residual = [i-j for i,j in zip(y_1,y_test)]
        #mean,std=norm.fit(residual)
        #print(mean,std,i,j)

0.4552326434757082 0.46006419242972035 5 7


In [19]:
check = Xmax>300
check2 = Xmax<900
new_check= []
for i,j in zip(check,check2):
    if i and j:
        new_check.append(True)
    else:
        new_check.append(False)
    
mask = regr_1.predict(input_variable)
mask_convert_pre = []
for i in mask:
    if i>=0.5:
        mask_convert_pre.append(True)
    else:
        mask_convert_pre.append(False)

mask_convert = []
for i,j in zip(new_check,mask_convert_pre):
    if i and j:
        mask_convert.append(True)
    else:
        mask_convert.append(False)

In [21]:
print(np.corrcoef(np.array([A[mask_convert],
                            D[mask_convert],
                            N[mask_convert],
                            S125[mask_convert],
                            beta[mask_convert],
                            zenith[mask_convert],
                            Xmax[mask_convert],
                            chi2[mask_convert],
                            red[mask_convert],
                            mass[mask_convert],
                            goodness[mask_convert]])))

[[ 1.          0.13216003 -0.06992996  0.07621445  0.05489924  0.15042306
   0.09380134  0.15893839 -0.00453584 -0.07362787 -0.17353523]
 [ 0.13216003  1.         -0.02965211 -0.51452004  0.00223243 -0.13862801
  -0.00464256  0.10350347 -0.06472633 -0.15834258 -0.04387069]
 [-0.06992996 -0.02965211  1.          0.05188534 -0.04363735 -0.04930809
  -0.04757658  0.63551711  0.02215767  0.06987297  0.017277  ]
 [ 0.07621445 -0.51452004  0.05188534  1.          0.11606144  0.37025016
   0.28211679 -0.07263374  0.13233394  0.05243804 -0.09357577]
 [ 0.05489924  0.00223243 -0.04363735  0.11606144  1.          0.26102807
   0.22185904 -0.0410097  -0.00821978 -0.19875661  0.05436811]
 [ 0.15042306 -0.13862801 -0.04930809  0.37025016  0.26102807  1.
  -0.05258422 -0.03980583 -0.01332203  0.01897163  0.09013254]
 [ 0.09380134 -0.00464256 -0.04757658  0.28211679  0.22185904 -0.05258422
   1.         -0.01587168  0.05792182 -0.65618207 -0.06744687]
 [ 0.15893839  0.10350347  0.63551711 -0.07263374

In [145]:
mask_new = []
import random
for i,j in zip(mask_convert,mass):
    if (i and j==1) and random.random()<0.15:
        mask_new.append(False)
    else:
        mask_new.append(i)

In [146]:
input_variable2 = np.array([np.append(i,j) for i,j in zip(D[mask_new],beta[mask_new])])
output_new = mass[mask_new]

In [147]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.2
X_train, X_test, y_train_1, y_test_1 = train_test_split(input_variable2,output_new , test_size=test_size, random_state=seed)

In [166]:
from scipy.stats import norm
for i in range(1,100):
    for j in [15,18]:
        rng = np.random.RandomState(1)
        regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=j,criterion='friedman_mse'),
                                        n_estimators=i, random_state=rng,loss='square')
        regr_2.fit(X_train,y_train_1)
    
        new_1= regr_2.predict(X_test)
        new_2 = regr_2.predict(X_train)
        square = [(i-j)**2.0 for i,j in zip(new_1,y_test_1)]
        square2 = [(i-j)**2.0 for i,j in zip(new_2,y_train_1)]
        #print(np.mean(square),np.mean(square2),i,j)
        mass_predict = []
        for k in new_1:
            if k <=2.5:
                mass_predict.append(1)
            elif k>2.5:
                mass_predict.append(4)
                
        mass_predict2 = []
        for k in new_2:
            if k <=2.5:
                mass_predict2.append(1)
            elif k>2.5:
                mass_predict2.append(4)
                
        from sklearn.metrics import confusion_matrix
        cm=confusion_matrix(y_test_1,mass_predict)
        cm2 = confusion_matrix(y_train_1,mass_predict2)
        print(cm[0][0]/np.sum(cm[0]),cm[1][1]/np.sum(cm[1]),cm2[0][0]/np.sum(cm2[0]),cm2[1][1]/np.sum(cm2[1]),i,j)

0.7127461260631481 0.5385925751879699 0.7533991323841965 0.5846176448982591 1 15
0.6924734941162763 0.540296052631579 0.7642588872390602 0.6170131491956218 1 18
0.7127461260631481 0.5385925751879699 0.7533991323841965 0.5846176448982591 2 15
0.6924734941162763 0.540296052631579 0.7642588872390602 0.6170131491956218 2 18
0.7128626354421531 0.5503994360902256 0.742160887413748 0.5850584000587673 3 15
0.7185715950133986 0.5453477443609023 0.7663260255626401 0.5996327040329097 3 18
0.7304555516719096 0.5386513157894737 0.7569365592337031 0.5730845515316242 4 15
0.7245135733426541 0.5360667293233082 0.7746236934812356 0.5959744362006906 4 18
0.6807060468367704 0.5898143796992481 0.7027979153929018 0.6172629104532432 5 15
0.7534661540253991 0.5152725563909775 0.7790927883075669 0.5495923014765298 5 18
0.7161248980542934 0.556625939849624 0.7370949427897633 0.586057445089253 6 15
0.7383199347547478 0.5316611842105263 0.7792529187410836 0.5784617644898259 6 18
0.7019107538156821 0.570429981203

In [176]:
rng = np.random.RandomState(1)
regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=15,criterion='friedman_mse'),
                            n_estimators=49, random_state=rng,loss='square')
regr_2.fit(X_train,y_train_1)
    
new_1= regr_2.predict(X_test)
new_2 = regr_2.predict(X_train)
mse = [(m-n)**2.0 for m,n in zip(new_1,y_test_1)]
mse2 = [(m-n)**2.0 for m,n in zip(new_2,y_train_1) ]
print(np.mean(mse),np.mean(mse2),i,j)

2.247797792804042 2.244624531724793 2.4955840831231413 18


In [177]:
mass_predict = []
for i in new_1:
    if i <2.5:
        mass_predict.append(1)
    else:
        mass_predict.append(4)

In [178]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test_1,mass_predict))
cm=confusion_matrix(y_test_1,mass_predict)

[[ 1822 15344]
 [  973 16051]]


In [179]:
print(cm[0][0]/np.sum(cm[0]),cm[1][1]/np.sum(cm[1]))

0.10614004427356402 0.9428453947368421


In [180]:
rng = np.random.RandomState(1)
regr_3 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=18,criterion='friedman_mse'),
                            n_estimators=32, random_state=rng,loss='square')
regr_3.fit(X_train,y_train_1)
    
new_3= regr_3.predict(X_test)
new_4 = regr_3.predict(X_train)
mse = [(m-n)**2.0 for m,n in zip(new_3,y_test_1)]
mse2 = [(m-n)**2.0 for m,n in zip(new_4,y_train_1) ]
print(np.mean(mse),np.mean(mse2),i,j)

2.2457277284546606 2.2373313976472424 2.5003564012768464 18


In [181]:
mass_predict1 = []
for i in new_3:
    if i <2.5:
        mass_predict1.append(1)
    else:
        mass_predict1.append(4)

In [182]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test_1,mass_predict1))
cm=confusion_matrix(y_test_1,mass_predict1)

[[14841  2325]
 [11009  6015]]


In [183]:
print(cm[0][0]/np.sum(cm[0]),cm[1][1]/np.sum(cm[1]))

0.864557846906676 0.3533247180451128
